In [ ]:
from VectorDataBase import WeaviateDB
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from Saul_LLM import Legal_LLM

In [ ]:
class RAGDemo:
    def __init__(self, collection_names=['Uk', 'Whales']):
        """
        Initializes the RAGDemo object.
        
        Args:
            collection_names (list, optional): A list of collection names. Defaults to ['Uk'].
        """
        self.vector_db = WeaviateDB(collection_names=collection_names)
        self.llm = Legal_LLM()

    def add_text(self, collection_name, text, metadata=None):
        """
        Adds text data to a specified collection in the Weaviate database.
        
        Args:
            collection_name (str): The name of the collection in the database.
            text (str): The text data to be added.
            metadata (dict): Additional metadata associated with the text.
        """
        self.vector_db.add_text_to_db(
            collection_name=collection_name,
            text=text,
            metadata=metadata
        )

    def query(self, collection_name, query, k=1):
        """
        Performs a RAG query on the specified collection using the Ollama LLM.
        
        Args:
            collection_name (str): The name of the collection in the database.
            query (str): The query to search for similar documents.
            k (int, optional): The number of documents to return. Defaults to 3.
        
        Returns:
            None
        
        Prints the similarity score and the content of the top k documents that match the query.
        """
        current_db = self.vector_db.vector_stores[collection_name]
        
        # Create a retriever for the current database
        retriever = current_db.as_retriever(
            search_kwargs={"k": k})

        # Function to format documents into a single context string
        def format_docs(docs):
            print(f'The retrieved documents are:')
            for idx,doc in enumerate(docs):
                print(f'{idx} - Content: {doc.page_content[:50]}...')
            print()
            return "\n\n".join(doc.page_content for doc in docs)
        
        retrieved_docs = retriever.get_relevant_documents(query)
        context = format_docs(retrieved_docs)
        
        response = self.llm.chat(context={context},
                                 query={query}, max_new_tokens=512)
        print(response)

In [ ]:
app = RAGDemo(
    ['Uk', 'Wales']
)

In [ ]:
app.vector_db.validate_collection()

In [ ]:
# app.vector_db.delete_collection('Whales')
# app.vector_db.delete_collection('Uk')

In [ ]:
app.add_text(
    collection_name='Uk',
    text='Artificial Intelligence revolutionizes industries by enhancing data analysis, automating tasks, and providing innovative solutions to complex problems effortlessly.',
)

In [ ]:
app.add_text(
    collection_name='Wales',
    text='Maintaining good health require - Cluster Status:Trues balanced nutrition, regular exercise, adequate sleep, and mindfulness to promote overall well-being and prevent chronic illnesses.',
)

In [ ]:
app.add_text(
    collection_name='Uk',
    text='Legality ensures order, fairness, and justice within a society by establishing rules and consequences that govern individual behavior. It provides a framework for resolving disputes, protecting rights, and maintaining public safety, which fosters trust and cooperation among citizens and institutions.',
)

In [ ]:
# Perform a RAG query with a sample query
app.query(
    collection_name='Uk',
    query="How does Artificial Intelligence revolutionize industries and enhance data analysis?"
)

In [ ]:
# Perform a RAG query with a sample query
app.query(
    collection_name='Wales',
    query="In what ways can AI automate tasks and provide innovative solutions to complex problems?"
)

>_

In [ ]:
# Perform a RAG query with a sample query
app.query(
    collection_name='Uk',
    query="What are the key components required to maintain good health and overall well-being?"
)

In [ ]:
# Perform a RAG query with a sample query
app.query(
    collection_name='Whales',
    query="How does legality ensure order, fairness, and justice within a society?"
)